# Rover development notebook (draft/development)

This notebook is for development and testing of the Rover. It is meant to be used with the Rover connected to USB during development.

- The communication with vehicle using [The Simple Communication Protocol for Embedded Applications](https://github.com/nhorro/simpleprotocol).

In [1]:
%load_ext autoreload
%autoreload 2

## Vehicle operation

![img](../Documentation/Assets/RoverInTestBenchTopView.jpeg)

### Connection to vehicle

Find which port is the vehicle connected to.
In linux, usually it is ttyACM0, ttyACM1, ...

In [2]:
!ls /dev/ttyACM*

/dev/ttyACM0


In [3]:
from rover.vehicleif import VehicleIF
vehicle = VehicleIF("/dev/ttyACM0", 115200,debug=False)
vehicle.capture_path = "./captures"

For a quick health check, see current telemetry state:

In [4]:
vehicle.print_telemetry()

General Telemetry
   TelemetryCycle: 0x00001070
   OnBoardTime: 0x00403571
   ReceivedPackets: 0x00000002
   LedControlState: 0x00000000
   LastCommandOpcode: 0x00000000
   LastCommandResult: 0x00000001
   LasOsResult1: 0x00000000
   LasOsResult2: 0x00000000
Motor Telemetry
   Throttle1: 0.0
   Throttle2: 0.0
   Tachometer1: 28
   Tachometer2: 145
   Tachometer3: 20
   Tachometer4: 12
   MeasuredSpeed1: 0.0
   MeasuredSpeed2: 0.0
   MeasuredSpeed3: 0.0
   MeasuredSpeed4: 0.0
   SetpointSpeed1: 0.0
   SetpointSpeed2: 0.0
   StatusFlags: 0


For a quick command test, turn on and off led.

In [8]:
# Led on
vehicle.control_led(0x00000001)

Sent: 0x 40 3C 05 00 00 00 00 01 A2 0A  (10 bytes)


In [9]:
# Led off
vehicle.control_led(0x00000000)

Sent: 0x 40 3C 05 00 00 00 00 00 93 0A  (10 bytes)


## Telemetry analysis

The following tests use logged telemetry. Telemetry is received and saved to CSV files. Each telemetry report type is saved to its own file.

The following methods start and stop the logging of all received reports:

- `start_telemetry_csv_logging()`
- `stop_telemetry_csv_logging()`

In [12]:
vehicle.start_telemetry_csv_logging()

In [14]:
vehicle.stop_telemetry_csv_logging()

CSVs will be saved to the `capture_path` variable of the vehicle interface.

In [15]:
!ls {vehicle.capture_path}

tctm.telemetry.GeneralTelemetry.csv  tctm.telemetry.MotorControlTelemetry.csv


In [19]:
import os
import pandas as pd
import matplotlib.pyplot as plt

general_tm_df = pd.read_csv(os.path.join(vehicle.capture_path,"tctm.telemetry.GeneralTelemetry.csv"))
general_tm_df

,TelemetryCycle,OnBoardTime,ReceivedPackets,LedControlState,LastCommandOpcode,LastCommandResult,LasOsResult1,LasOsResult2
0,4531,4530985,7,0,0,1,0,0
1,4532,4531985,7,0,0,1,0,0
2,4533,4532985,7,0,0,1,0,0
3,4534,4533985,7,0,0,1,0,0
4,4535,4534985,7,0,0,1,0,0
5,4536,4535985,7,0,0,1,0,0
6,4537,4536985,7,0,0,1,0,0
7,4538,4537985,7,0,0,1,0,0
8,4539,4538985,7,0,0,1,0,0
9,4540,4539985,7,0,0,1,0,0


In [21]:
motor_control_tm_df = pd.read_csv(os.path.join(vehicle.capture_path,"tctm.telemetry.MotorControlTelemetry.csv"))
motor_control_tm_df

,Throttle1,Throttle2,Tachometer1,Tachometer2,Tachometer3,Tachometer4,MeasuredSpeed1,MeasuredSpeed2,MeasuredSpeed3,MeasuredSpeed4,SetpointSpeed1,SetpointSpeed2,StatusFlags
0,0.0,0.0,28,145,20,12,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,28,145,20,12,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,28,145,20,12,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,28,145,20,12,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,28,145,20,12,0.0,0.0,0.0,0.0,0.0,0.0,0
5,0.0,0.0,28,145,20,12,0.0,0.0,0.0,0.0,0.0,0.0,0
6,0.0,0.0,28,145,20,12,0.0,0.0,0.0,0.0,0.0,0.0,0
7,0.0,0.0,28,145,20,12,0.0,0.0,0.0,0.0,0.0,0.0,0
8,0.0,0.0,28,145,20,12,0.0,0.0,0.0,0.0,0.0,0.0,0
9,0.0,0.0,28,145,20,12,0.0,0.0,0.0,0.0,0.0,0.0,0


### Sending commands

In [ ]:
vehicle.set_motor_throttles(0.0,0.51,0x3)

In [ ]:
vehicle.set_motor_throttles(1.0,1.0,0x3)

In [ ]:
# Stop motors
vehicle.set_motor_throttles(0.0,0.,0x3)
vehicle.set_motor_speeds(0.0,0.,0x3)

In [ ]:
vehicle.set_motor_speeds(0.01,0.01,0x3)

In [ ]:
vehicle.set_pid_parameters(0.1,5.3,5.0)

In [ ]:
vehicle.set_motor_control_mode(0)   

## Checking received telemetry

In [ ]:
vehicle.print_telemetry()

## Teardown

In [20]:
vehicle.shutdown()

# Work in progress

<hr/>